In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, mean_absolute_error, r2_score
from sklearn.preprocessing import label_binarize


In [2]:
base_path = os.path.join(os.getcwd().split('pretrain_comparison/evaluation')[0], 'pretrain_comparison/output/results')
files_list = list(Path(base_path).rglob('*.npy'))

In [3]:

rows = []

for file in files_list:
    if 'sleep_stage_and_age_results.npy' not in str(file):
        continue

    name = str(file).split('/')[-2]
    print(f"Calculating sleep stage and age results for {name}")
    results = np.load(file, allow_pickle=True).item()

    age_predictions = results['age_predictions']
    age_targets = results['age_targets']
    sleep_predictions = results['sleep_predictions']
    sleep_targets = results['sleep_targets']

    classes = np.unique(sleep_targets)
    sleep_targets_bin = label_binarize(sleep_targets, classes=classes)
    sleep_predictions_bin = label_binarize(sleep_predictions, classes=classes)

    class_aucs = []
    for i in range(len(classes)):
        try:
            auc = roc_auc_score(sleep_targets_bin[:, i], sleep_predictions_bin[:, i])
            class_aucs.append(auc)
        except ValueError:
            class_aucs.append(None)

    mean_auc = np.nanmean(class_aucs)

    age_mae = mean_absolute_error(age_targets, age_predictions)
    age_r2 = r2_score(age_targets, age_predictions)
    age_pearson = np.corrcoef(age_targets, age_predictions)[0, 1]
    age_spearman = np.corrcoef(np.argsort(age_targets), np.argsort(age_predictions))[0, 1]

    row = {
        'name': name,
        'sleep_stage_auc': mean_auc,
        'W_auc': class_aucs[0] if len(class_aucs) > 0 else None,
        'N1_auc': class_aucs[1] if len(class_aucs) > 1 else None,
        'N2_auc': class_aucs[2] if len(class_aucs) > 2 else None,
        'N3_auc': class_aucs[3] if len(class_aucs) > 3 else None,
        'REM_auc': class_aucs[4] if len(class_aucs) > 4 else None,
        'age_mae': age_mae,
        'age_r2': age_r2,
        'age_pearson': age_pearson,
        'age_spearman': age_spearman
    }
    rows.append(row)

df = pd.DataFrame(rows, columns=[
    'name', 'sleep_stage_auc', 'W_auc', 'N1_auc', 'N2_auc', 'N3_auc', 'REM_auc',
    'age_mae', 'age_r2', 'age_pearson', 'age_spearman'
])


Calculating sleep stage and age results for CL_pairwise_epochs_36


In [4]:
df

,name,sleep_stage_auc,W_auc,N1_auc,N2_auc,N3_auc,REM_auc,age_mae,age_r2,age_pearson,age_spearman
0,CL_pairwise_epochs_36,0.5,0.5,0.5,0.5,0.5,0.5,0.424168,-5.930733,0.387248,0.285714


In [5]:
df.to_csv(os.path.join(base_path, 'sleep_stage_and_age_results.csv'), index=False)